In [ ]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import alchemy_tools_pyscf as atp
import numpy as np
import qml
import utils_qm as ut

In [ ]:
lam_vals = np.arange(2, 54, 2)/52
basis = 'def2-tzvp'
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')
alchemical_potentials = []


for lam in lam_vals:
    num_elec = atp.get_num_elec(lam, com.nuclear_charges.sum())
    print('preparing input')
    deltaZ, includeonly, mol = atp.prepare_input(com.coordinates, com.nuclear_charges, num_elec, basis)
    print('Doing SCF calculation')
    dm, e_tot = atp.make_apdft_calc(deltaZ, includeonly, mol, method = "HF")
    print('Calculating alchemical potentials')
    alchpots_lambda = atp.calculate_alchpot(dm, includeonly, mol)

    alchemical_potentials.append(alchpots_lambda)

In [ ]:
alchemical_potentials = np.array(alchemical_potentials)

In [ ]:
base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
ut.save_obj(alchemical_potentials, base+'alch_pots_fine')

#### Plot results

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
alchpots = ut.load_obj(base+'alch_pots_fine')
lam_vals = np.arange(2, 54, 2)/52

In [ ]:
fig, ax = plt.subplots(1,1)
color_dict = {1.0:'tab:blue', 6.0:'tab:green', 7.0:'tab:orange', 8.0:'tab:red'}
for i in range(len(com.nuclear_charges)):
    
    ax.plot(lam_vals, alchpots[:,i], '-o', label = com.nuclear_charges[i], color=color_dict[com.nuclear_charges[i]])
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:
alchpots = np.concatenate((np.zeros( (1, len(alchpots[0])) ), alchpots), axis = 0)
lam_vals = np.concatenate((np.zeros(1), lam_vals))

In [ ]:
lam_vals[11:]*52

#### Compare integrals

In [ ]:
av_fine = atp.calculate_average_alchpots(alchpots[11:], lam_vals[11:], 'trapz')

In [ ]:
av_fine

In [ ]:
# load reference data from different_resevoir_pyscf.ipynb
import utils_qm as ut
com = qml.Compound(xyz='/home/misa/datasets/qm9/dsgdb9nsd_001212.xyz')

base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
alchs_ref = ut.load_obj(base+'alch_pots')
alch_pots_reference = -np.array(alchs_ref)
alch_pots_reference = np.concatenate((np.zeros( (1, len(alch_pots_reference[0])) ), alch_pots_reference), axis = 0)
lam_vals_ref = np.array([0, 22, 30, 38, 44, 52])/52

In [ ]:
alch_pots_reference[:,0]

In [ ]:
av_coarse = atp.calculate_average_alchpots(alch_pots_reference[1:], lam_vals_ref[1:], 'trapz')

In [ ]:
av_coarse

In [ ]:
av_fine - av_coarse

some potentials differ quite a lot (>100mHa) -> also small lambda values important

What is the best spacing (cost vs accuracy)?
- equally spaced -> simpsons rule
- compare every 2, 4, 6, 8, 10

#### Trapezoidal vs Simpsons

In [1]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')

import alchemy_tools_pyscf as atp
import numpy as np
import qml
import utils_qm as ut

In [2]:
base = '/home/misa/APDFT/prototyping/atomic_energies/results/different_resevoir/pyscf/com_001212/'
alchpots = ut.load_obj(base+'alch_pots_fine')
lam_vals = np.arange(2, 54, 2)/52
alchpots = np.concatenate((np.zeros( (1, len(alchpots[0])) ), alchpots), axis = 0)
lam_vals = np.concatenate((np.zeros(1), lam_vals))

In [3]:
av2_trapz = atp.calculate_average_alchpots(alchpots, lam_vals, 'trapz')
av2_simps = atp.calculate_average_alchpots(alchpots, lam_vals, 'simps')


In [4]:
av2_trapz - av2_simps

array([-0.00233282, -0.00427446, -0.00797825,  0.00076161,  0.00279349,
        0.00061471, -0.00249642,  0.0046545 , -0.00201606, -0.00204067,
        0.00325231,  0.00322552])

In [ ]:
len(lam_vals[::2])

In [5]:
av4_trapz = atp.calculate_average_alchpots(alchpots[::2], lam_vals[::2], 'trapz')
av4_simps = atp.calculate_average_alchpots(alchpots[::2], lam_vals[::2], 'simps', 'first')

In [6]:
av4_trapz - av4_simps

array([-0.01736299, -0.00661817, -0.01755904, -0.01258776, -0.00338907,
       -0.00872064, -0.00614399, -0.00632665, -0.0073069 , -0.00723812,
        0.00614106,  0.00616014])

In [7]:
av2_simps - av4_simps

array([-0.00803169,  0.01047966,  0.01435398, -0.01563419, -0.01456303,
       -0.01117948,  0.00384171, -0.02494466,  0.00075734,  0.00092454,
       -0.00686816, -0.00674194])